# RUNet (Robust U-Net) implementation  
**Authors:** Konrad Maciejczyk (276927), Robert Walery (24900)  
Wrocław University of Science and Technology, 2023-2024

Dataset: [*STL-10 Image Recognition Dataset*](https://www.kaggle.com/datasets/jessicali9530/stl10?resource=download)  
Reference paper: Hu, Xiaodan, et al. [*RUNet: A Robust UNet Architecture for Image Super-Resolution*](https://openaccess.thecvf.com/content_CVPRW_2019/papers/WiCV/Hu_RUNet_A_Robust_UNet_Architecture_for_Image_Super-Resolution_CVPRW_2019_paper.pdf)

---

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2
import datetime

if len(tf.config.list_physical_devices('GPU')) < 1:
  raise Exception("GPU device not found!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/DATA/stl_10.zip

## Data loading and data preprocessing
For our needs the training data is descaled half of it's original size (from 96x96 to 46x46).  
The dataset is splitted into training data (80% - 4000 images) and testing data (20% - 1000 images).

In [ ]:
input_folder = "./train_images/"
image_files = [f for f in os.listdir(input_folder) if f.endswith(".png") or f.endswith(".jpg")]
images_Y = [cv2.imread(os.path.join(input_folder, img)) for img in image_files]
images_X = [cv2.resize(img, (48, 48), interpolation=cv2.INTER_AREA) for img in images_Y]
images_X = [img / 255.0 for img in images_X]
images_Y_ = [img / 255.0 for img in images_Y]
images_X = np.array(images_X)
images_Y = np.array(images_Y)
X_train, X_test, Y_train, Y_test = train_test_split(images_X, images_Y, test_size=0.2, random_state=42)

input_size = 48
print(X_train.shape)
print(Y_test.shape)
print(type(X_train))

(1600, 48, 48, 3)
(400, 96, 96, 3)
<class 'numpy.ndarray'>


## RUNet architecture  
### Auxiliary functions for repetetive fragments

In [ ]:
# pixel shuffle
def pixel_shuffle(scale):
    if scale > 1:
        return lambda x: tf.nn.depth_to_space(x, scale)
    else:
        return lambda x:x

def insert_downsampling_block(x_inp, filters, kernel_size=(3, 3), padding="same", strides=1,r=False):
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if r:
        x_inp = tf.keras.layers.Conv2D(filters,(1,1), padding=padding, strides=strides)(x_inp)
    x = tf.keras.layers.Add()([x,x_inp])
    return x

def insert_bottleneck(x_inp,filters, kernel_size=(3, 3), padding="same", strides=1):
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x_inp)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def insert_upscaling_block(x_inp,skip,filters, kernel_size=(3, 3), padding="same", strides=1,upscale_factor=2):
    x = pixel_shuffle(scale=upscale_factor)(x_inp)
    x = tf.keras.layers.Concatenate()([x, skip])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
input = tf.keras.layers.Input(shape = (input_size, input_size, 3))
down_1 = tf.keras.layers.Conv2D(64,(7,7), padding="same", strides=1)(input)
down_1 = tf.keras.layers.BatchNormalization()(down_1)
down_1 = tf.keras.layers.Activation('relu')(down_1)

down_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(down_1)
down_2 = insert_downsampling_block(down_2, 64)
down_2 = insert_downsampling_block(down_2, 64)
down_2 = insert_downsampling_block(down_2, 64)
down_2 = insert_downsampling_block(down_2, 128, r=True)

down_3 = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2)(down_2)
down_3 = insert_downsampling_block(down_3, 128)
down_3 = insert_downsampling_block(down_3, 128)
down_3 = insert_downsampling_block(down_3, 128)
down_3 = insert_downsampling_block(down_3, 256, r=True)

down_4 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(down_3)
down_4 = insert_downsampling_block(down_4,256)
down_4 = insert_downsampling_block(down_4,256)
down_4 = insert_downsampling_block(down_4,256)
down_4 = insert_downsampling_block(down_4,256)
down_4 = insert_downsampling_block(down_4,256)
down_4 = insert_downsampling_block(down_4,512,r=True)

down_5 = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(down_4)
down_5 = insert_downsampling_block(down_5,512)
down_5 = insert_downsampling_block(down_5,512)
down_5 = tf.keras.layers.BatchNormalization()(down_5)
down_5 = tf.keras.layers.Activation('relu')(down_5)

bn_1 = insert_bottleneck(down_5, 1024)
bn_2 = insert_bottleneck(bn_1, 512)

up_1 = insert_upscaling_block(bn_2,down_5, 512,upscale_factor=1)
up_2 = insert_upscaling_block(up_1,down_4, 384,upscale_factor=2)
up_3 = insert_upscaling_block(up_2,down_3, 256,upscale_factor=2)
up_4 = insert_upscaling_block(up_3,down_2, 96,upscale_factor=2)

up_5 = pixel_shuffle(scale=2)(up_4)
up_5 = tf.keras.layers.Concatenate()([up_5,down_1])
up_5 = tf.keras.layers.Conv2D(99,(3,3), padding="same", strides=1)(up_5)
up_5 = tf.keras.layers.Activation('relu')(up_5)
up_5 = tf.keras.layers.Conv2D(99,(3,3), padding="same", strides=1)(up_5)
up_5 = tf.keras.layers.Activation('relu')(up_5)

up_conv4 = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(2,2), strides=(2,2), padding='same')(up_5)
conv8_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(up_conv4)
conv8_2 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(conv8_1)

output = tf.keras.layers.Conv2D(3,(1,1), padding="same")(conv8_2)
model = tf.keras.models.Model(input, output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 48, 48, 3)]          0         []                            
                                                                                                  
 conv2d_419 (Conv2D)         (None, 48, 48, 64)           9472      ['input_11[0][0]']            
                                                                                                  
 batch_normalization_335 (B  (None, 48, 48, 64)           256       ['conv2d_419[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_279 (Activation  (None, 48, 48, 64)           0         ['batch_normalization_33

## Metrics used as loss functions
**PSNR** - In image evaluation, PSNR (Peak Signal-to-Noise Ratio) quantifies the ratio between the maximum possible power of the image and the noise introduced during compression, giving an indication of how closely the reconstructed image matches the original in terms of fidelity.

**SSIM** - SSIM stands for Structural Similarity Index Measure. It's a metric used to evaluate the similarity between two images, measuring not just the resemblance in pixel values but also accounting for perceived changes in structural information, luminance, and contrast. SSIM assesses the similarity in structure by considering local patterns in the images and human visual perception characteristics, providing a more comprehensive assessment of image quality than traditional pixel-based metrics.

In [63]:
def psnr(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)

def ssim(y_true,y_pred):
    return tf.image.ssim(y_true,y_pred,1.0)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[psnr, ssim])

In [ ]:
model.fit(X_train, Y_train, epochs=50, batch_size=128, validation_split=0.2)

timestamp = datetime.datetime.now()
model.save(f'{timestamp.day}_{timestamp.month}__{timestamp.hour}_{timestamp.minute}.h5')

10/10 [==============================] - 4s 439ms/step - loss: 220.6476 - psnr: -22.9478 - val_loss: 240.2100 - val_psnr: -23.3955


In [ ]:
!mv ./TEST /content/drive/MyDrive/